In [443]:
import pandas as pd
import re
import datetime
import numpy as np
from dateutil.relativedelta import relativedelta
from datetime import date
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer, TransformedTargetRegressor
from sklearn.metrics import mean_squared_error, r2_score, make_scorer

In [444]:
claims = pd.read_parquet("claims.parquet")

In [448]:
policies = pd.read_excel("policies.xlsx",engine = "openpyxl")

In [451]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [452]:
policies['pol'].value_counts()

f976a747    4
1edad92c    4
206daa37    4
03d18350    4
bd8d5692    4
           ..
c0c31cc6    1
66e20e0c    1
5df4372c    1
5d3eca51    1
7908e09d    1
Name: pol, Length: 1000, dtype: int64

In [486]:
def read_files(parquet_file, csv_file, xlsx_file):
    claims = pd.read_parquet(parquet_file)
    policies = pd.read_excel(xlsx_file, engine = "openpyxl")
    properties = pd.read_csv(csv_file)
    claims_clean = date_process(claims, "start_date")
    policies_start_clean = date_process(policies, "start")
    policies_clean = date_process(policies_start_clean, "end")
    final_data = data_process(claims_clean, policies_clean, properties)
    print(final_data)

In [487]:
def date_process(df, column):
    df1 = df[column].str.extract('(?P<day>\d+)(?P<month>[A-Za-z]{3})(?P<year>.*)')
    df1['year'].replace(regex=True, inplace=True, to_replace=r'[^0-9]', value=r'')
    df2 = pd.concat([df, df1], axis=1)
    df2.drop(column, axis = 1, inplace = True)
    df2[column] = pd.to_datetime(df2['year'].astype(str)  + df2['month'] + df2['day'].astype(str), format='%Y%b%d', errors = 'coerce')
    df2.drop(['day', 'month', 'year'], axis = 1, inplace = True)
    return df2

In [488]:
def data_process(claims, policies, properties):
    claims.dropna(inplace = True)
    policies.dropna(inplace = True)
    policies['duration'] = policies['end'] - policies['start']
    policies['duration'] = (policies['duration']/ datetime.timedelta(days = 365)).astype(int)
    claims_grouped = claims.groupby(['property', 'pol', 'start_date']).count().reset_index()
    interim_result = pd.merge(claims_grouped, properties, left_on = "property", right_on = "prop_id", how = "inner")
    interim_result.drop(['pol_x', 'property'], inplace = True, axis = 1)
    interim_result.rename(columns = {'pol_y': 'pol', 'amount': 'claims'}, inplace = True)
    result = pd.merge(interim_result, policies, left_on = ['pol', 'start_date'], right_on = ['pol', 'start'], how = "inner")
    result.drop(['end', 'start', 'prop_id'], inplace = True, axis = 1)
    result['exposure'] = (result['sqft']/1000) * result['duration']
    result['claim_frequency'] = result['claims']/result['exposure']
    return result

In [489]:
read_files("claims.parquet", "properties.csv", "policies.xlsx")

      start_date  claims       pol state   sqft  age  duration  exposure  \
0     2015-11-06       5  ae71135b    AZ  88972   83         1    88.972   
1     2015-11-06       1  ae71135b    OH  43434   11         1    43.434   
2     2015-11-06       8  ae71135b    FL  66035    8         1    66.035   
3     2015-11-06       5  ae71135b    AZ  18145   59         1    18.145   
4     2015-11-06       3  ae71135b    AZ  74404   50         1    74.404   
...          ...     ...       ...   ...    ...  ...       ...       ...   
14237 2018-07-06      11  2cde1518    OH  65374   39         1    65.374   
14238 2018-07-06       1  2cde1518    AZ  84879   45         1    84.879   
14239 2018-07-06       6  2cde1518    AZ  65986   25         1    65.986   
14240 2018-07-06      10  2cde1518    OH  16746   85         1    16.746   
14241 2018-07-06       3  2cde1518    AZ  24166   47         1    24.166   

       claim_frequency  
0             0.056197  
1             0.023023  
2           

In [417]:
df_train, df_test = train_test_split(result_final, test_size=0.2, random_state=0)

In [418]:
numeric_transformer = Pipeline(
    steps = [
        ("scaler", StandardScaler())
    ]
)

categorical_transformer = Pipeline(
    steps=[
        ("onehot", OneHotEncoder(handle_unknown="ignore")),
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, ['sqft', 'age', 'exposure', 'claims_no']),
        ("cat", categorical_transformer, ['state']),
    ]
)

pipe = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("regressor", Ridge(alpha = 1e-6)),
    ]
)

In [420]:
X_train = result_final.drop(columns = ['claim_frequency'])
y_train = result_final['claim_frequency']

In [423]:
from sklearn.linear_model import Ridge

pd.DataFrame(cross_validate(
    pipe, X_train, y_train, return_train_score=True, 
    scoring="mean_absolute_error"
))


ValueError: 'mean_absolute_error' is not a valid scoring value. Use sorted(sklearn.metrics.SCORERS.keys()) to get valid options.